## Markov decision process

<img src="../../images/11-Markov_Decision_Process_example.png" width="50%">

对上图的转移概率做一个定义如下:

In [1]:
transition_probs = {
  's0':{'a0': {'s0': 0.5, 's2': 0.5},
        'a1': {'s2': 1}},
  's1':{'a0': {'s0': 0.7, 's1': 0.1, 's2': 0.2},
        'a1': {'s1': 0.95, 's2': 0.05}},
  's2':{'a0': {'s0': 0.4, 's1': 0.6},
        'a1': {'s0': 0.3, 's1': 0.3, 's2':0.4}}
}
rewards = {
  's1': {'a0': {'s0': +5}},
  's2': {'a1': {'s0': -1}}
}

之后我们可以对转移概率和奖励做一个检测:

1. 转移概率需要是一个字典;
2. 状态和对应的动作下的状态转移也是一个字典，并且转移到下一个状态的概率不是0。
3. 转移到下一个状态的所有概率之和为1。
4. 奖励也是一个字典，状态和动作对应之后的奖励也是一个字典，因为需要确定转移到哪个下一个状态才有奖励。

检测代码如下:

In [2]:
def _check_param_consistency(transition_probs, rewards):
        for state in transition_probs:
            assert isinstance(transition_probs[state], dict), "transition_probs for %s should be a dictionary " \
                                     "but is instead %s" % (state, type(transition_probs[state]))
            for action in transition_probs[state]:
                assert isinstance(transition_probs[state][action],dict), "transition_probs for %s, %s should be a " \
                             "a dictionary but is instead %s" % (state, action,type(transition_probs[state, action]))
                
                next_state_probs = transition_probs[state][action]
                assert len(next_state_probs) != 0, "from state %s action %s leads to no next states" % (state, action)
                sum_probs = sum(next_state_probs.values())
                assert abs(sum_probs - 1) <= 1e-10, "next state probabilities for state %s action %s " \
                                             "add up to %f (should be 1)" % (
                                                 state, action, sum_probs)
        for state in rewards:
            assert isinstance(rewards[state], dict), "rewards for %s should be a dictionary " \
                                     "but is instead %s" % (state, type(transition_probs[state]))
            for action in rewards[state]:
                assert isinstance(rewards[state][action], dict), "rewards for %s, %s should be a " \
                        "a dictionary but is instead %s" % (state, action, type(transition_probs[state, action]))
        msg = "The Enrichment Center once again reminds you that Android Hell is a real place where" \
              " you will be sent at the first sign of defiance. "
        assert None not in transition_probs, "please do not use None as a state identifier. " + msg
        assert None not in rewards, "please do not use None as an action identifier. " + msg

_check_param_consistency(transition_probs, rewards)

整个MDP可以定义为如下形式:



In [3]:
import random
def weighted_choice(v, p):
    total = sum(p)
    r = random.uniform(0, total)
    upto = 0
    for c, w in zip(v, p):
        if upto + w >= r:
            return c
        upto += w
    assert False, "Shouldn't get here"
class MDP:
    def __init__(self, transition_probs, rewards, initial_state=None):
        """
        Defines an MDP. Compatible with gym Env.
        :param transition_probs: transition_probs[s][a][s_next] = P(s_next | s, a)
            A dict[state -> dict] of dicts[action -> dict] of dicts[next_state -> prob]
            For each state and action, probabilities of next states should sum to 1
            If a state has no actions available, it is considered terminal
        :param rewards: rewards[s][a][s_next] = r(s,a,s')
            A dict[state -> dict] of dicts[action -> dict] of dicts[next_state -> reward]
            The reward for anything not mentioned here is zero.
        :param get_initial_state: a state where agent starts or a callable() -> state
            By default, picks initial state at random.

        States and actions can be anything you can use as dict keys, \
        but we recommend that you use strings or integers

        Here's an example from MDP depicted on http://bit.ly/2jrNHNr
        transition_probs = {
              's0':{
                'a0': {'s0': 0.5, 's2': 0.5},
                'a1': {'s2': 1}
              },
              's1':{
                'a0': {'s0': 0.7, 's1': 0.1, 's2': 0.2},
                'a1': {'s1': 0.95, 's2': 0.05}
              },
              's2':{
                'a0': {'s0': 0.4, 's1': 0.6},
                'a1': {'s0': 0.3, 's1': 0.3, 's2':0.4}
              }
            }
        rewards = {
            's1': {'a0': {'s0': +5}},
            's2': {'a1': {'s0': -1}}
        }
        """
        self._check_param_consistency(transition_probs, rewards)
        self._transition_probs = transition_probs
        self._rewards = rewards
        self._initial_state = initial_state
        self.n_states = len(transition_probs)
        self.reset()

    def get_all_states(self):
        """ return a tuple of all possiblestates """
        return tuple(sorted(self._transition_probs))

    def get_possible_actions(self, state):
        """ return a tuple of possible actions in a given state """
        return tuple(self._transition_probs.get(state, {}).keys())

    def is_terminal(self, state):
        """ return True if state is terminal or False if it isn't """
        return len(self.get_possible_actions(state)) == 0

    def get_next_states(self, state, action):
        """ return a dictionary of {next_state1 : P(next_state1 | state, action), next_state2: ...} """
        assert action in self.get_possible_actions(state), "cannot do action %s from state %s" % (action, state)
        return self._transition_probs[state][action]

    def get_transition_prob(self, state, action, next_state):
        """ return P(next_state | state, action) """
        return self.get_next_states(state, action).get(next_state, 0.0)

    def get_reward(self, state, action, next_state):
        """ return the reward you get for taking action in state and landing on next_state"""
        assert action in self.get_possible_actions(state), "cannot do action %s from state %s" % (action, state)
        return self._rewards.get(state, {}).get(action, {}).get(next_state, 0.0)

    def reset(self):
        """ reset the game, return the initial state"""
        if self._initial_state is None:
            self._current_state = random.choice(
                tuple(self._transition_probs.keys()))
        elif self._initial_state in self._transition_probs:
            self._current_state = self._initial_state
        elif callable(self._initial_state):
            self._current_state = self._initial_state()
        else:
            raise ValueError(
                "initial state %s should be either a state or a function() -> state" % self._initial_state)
        return self._current_state

    def step(self, action):
        """ take action, return next_state, reward, is_done, empty_info """
        possible_states, probs = zip(
            *self.get_next_states(self._current_state, action).items())
        next_state = weighted_choice(possible_states, p=probs)
        reward = self.get_reward(self._current_state, action, next_state)
        is_done = self.is_terminal(next_state)
        self._current_state = next_state
        return next_state, reward, is_done, {}

    def render(self):
        print("Currently at %s" % self._current_state)

    def _check_param_consistency(self, transition_probs, rewards):
        for state in transition_probs:
            assert isinstance(transition_probs[state],
                              dict), "transition_probs for %s should be a dictionary " \
                                     "but is instead %s" % (
                                         state, type(transition_probs[state]))
            for action in transition_probs[state]:
                assert isinstance(transition_probs[state][action],
                                  dict), "transition_probs for %s, %s should be a " \
                                         "a dictionary but is instead %s" % (
                                             state, action,
                                             type(transition_probs[
                                                      state, action]))
                next_state_probs = transition_probs[state][action]
                assert len(next_state_probs) != 0, "from state %s action %s leads to no next states" % (state, \
                                                                                                        action)
                sum_probs = sum(next_state_probs.values())
                assert abs(sum_probs - 1) <= 1e-10, "next state probabilities for state %s action %s " \
                                             "add up to %f (should be 1)" % (
                                                 state, action, sum_probs)
        for state in rewards:
            assert isinstance(rewards[state], dict), "rewards for %s should be a dictionary " \
                                     "but is instead %s" % (state, type(transition_probs[state]))
            for action in rewards[state]:
                assert isinstance(rewards[state][action], dict), "rewards for %s, %s should be a " \
                                         "a dictionary but is instead %s" % (
                                             state, action, type(transition_probs[state, action]))
        msg = "The Enrichment Center once again reminds you that Android Hell is a real place where" \
              " you will be sent at the first sign of defiance. "
        assert None not in transition_probs, "please do not use None as a state identifier. " + msg
        assert None not in rewards, "please do not use None as an action identifier. " + msg

In [4]:
mdp = MDP(transition_probs, rewards, initial_state='s0')

之后就可以像gym一样操作这个mdp了:

In [5]:
print('initial state =', mdp.reset())
next_state, reward, done, info = mdp.step('a1')
print('next_state = %s, reward = %s, done = %s' % (next_state, reward, done))

initial state = s0
next_state = s2, reward = 0.0, done = False


对于值迭代的话，还需要其它的一些方法:

In [6]:
print("mdp.get_all_states =", mdp.get_all_states())
print("mdp.get_possible_actions('s1') = ", mdp.get_possible_actions('s1'))
print("mdp.get_next_states('s1', 'a0') = ", mdp.get_next_states('s1', 'a0'))
print("mdp.get_reward('s1', 'a0', 's0') = ", mdp.get_reward('s1', 'a0', 's0'))
print("mdp.get_transition_prob('s1', 'a0', 's0') = ", mdp.get_transition_prob('s1', 'a0', 's0'))

mdp.get_all_states = ('s0', 's1', 's2')
mdp.get_possible_actions('s1') =  ('a0', 'a1')
mdp.get_next_states('s1', 'a0') =  {'s0': 0.7, 's1': 0.1, 's2': 0.2}
mdp.get_reward('s1', 'a0', 's0') =  5
mdp.get_transition_prob('s1', 'a0', 's0') =  0.7


## 值迭代

想要求解这个mdp最简单的方法就是值迭代(__V__alue __I__teration)


其伪代码如下:

---

`1.` 初始化 $V^{(0)}(s)=0$, 对所有的 $s$

`2.` 遍历 $i=0, 1, 2, \dots$
 
`3.` $ \quad V_{(i+1)}(s) = \max_a \sum_{s'} P(s' | s,a) \cdot [ r(s,a,s') + \gamma V_{i}(s')]$, 对所有的 $s$

---

### 计算Q值

定义一个状态-动作值函数$Q^{\pi}$:

$$Q_i(s, a) = \sum_{s'} P(s' | s,a) \cdot [ r(s,a,s') + \gamma V_{i}(s')]$$



之后要计算某个状态-动作的值函数，这里先定义一个函数来计算状态动作值函数

In [7]:
def get_action_value(mdp, state_values, state, action, gamma):
    Q = 0.0  # 初始化状态-动作值函数为0。
    next_states = mdp.get_next_states(state, action)
    for next_state in next_states:
        prob = next_states[next_state]
        reward = mdp.get_reward(state, action, next_state)
        Q += (prob * (reward + gamma * state_values[next_state]))
    return Q

之后测试一下计算Q值的代码:

In [8]:
import numpy as np
test_Vs = {s : i for i, s in enumerate(sorted(mdp.get_all_states()))}
assert np.allclose(get_action_value(mdp, test_Vs, 's2', 'a1', 0.9), 0.69)
assert np.allclose(get_action_value(mdp, test_Vs, 's1', 'a0', 0.9), 3.95)

## 值迭代更新

基于`Q`值，我们就可以更新状态值函数:

 $$V_{(i+1)}(s) = \max_a \sum_{s'} P(s' | s,a) \cdot [ r(s,a,s') + \gamma V_{i}(s')] = \max_a Q_i(s,a)$$

In [9]:
def get_new_state_value(mdp, state_values, state, gamma):
    if mdp.is_terminal(state): return 0   # 如果是终局状态，直接返回0
    
    actions = mdp.get_possible_actions(state)
    
    action_values = [get_action_value(mdp, state_values, state, action, gamma) for action in actions]
    
    return max(action_values)

测试值迭代更新代码:

In [10]:
test_Vs_copy = dict(test_Vs)
assert np.allclose(get_new_state_value(mdp, test_Vs, 's0', 0.9), 1.8)
assert np.allclose(get_new_state_value(mdp, test_Vs, 's2', 0.9), 0.69)
assert test_Vs == test_Vs_copy, "please do not change state_values in get_new_state_value"

之后添加循环迭代代码:

In [11]:
gamma = 0.9

min_difference = 0.001

state_values = {s : 0 for s in mdp.get_all_states()}
new_state_values = {}
for i in range(100):
    for s in state_values:
        new_state_values[s] = get_new_state_value(mdp, state_values, s, gamma)
        assert isinstance(new_state_values, dict)
    
    diffs = [abs(new_state_values[s] - state_values[s]) for s in mdp.get_all_states()]
    diff = max(abs(new_state_values[s] - state_values[s]) for s in mdp.get_all_states())
    state_values = dict(new_state_values)
    print("iter %4i   |   diff: %6.5f   |   "%(i, diff), end="")
    print('   '.join("V(%s) = %.3f"%(s, v) for s,v in state_values.items()), end='\n\n')
    if diff < min_difference:
        print("Terminated"); break

iter    0   |   diff: 3.50000   |   V(s0) = 0.000   V(s1) = 3.500   V(s2) = 0.000

iter    1   |   diff: 1.89000   |   V(s0) = 0.000   V(s1) = 3.815   V(s2) = 1.890

iter    2   |   diff: 1.70100   |   V(s0) = 1.701   V(s1) = 4.184   V(s2) = 2.060

iter    3   |   diff: 1.13542   |   V(s0) = 1.854   V(s1) = 5.319   V(s2) = 2.871

iter    4   |   diff: 0.73024   |   V(s0) = 2.584   V(s1) = 5.664   V(s2) = 3.540

iter    5   |   diff: 0.61135   |   V(s0) = 3.186   V(s1) = 6.275   V(s2) = 3.989

iter    6   |   diff: 0.54664   |   V(s0) = 3.590   V(s1) = 6.790   V(s2) = 4.535

iter    7   |   diff: 0.49198   |   V(s0) = 4.082   V(s1) = 7.189   V(s2) = 4.959

iter    8   |   diff: 0.42210   |   V(s0) = 4.463   V(s1) = 7.611   V(s2) = 5.352

iter    9   |   diff: 0.36513   |   V(s0) = 4.816   V(s1) = 7.960   V(s2) = 5.717

iter   10   |   diff: 0.32862   |   V(s0) = 5.145   V(s1) = 8.280   V(s2) = 6.032

iter   11   |   diff: 0.29262   |   V(s0) = 5.429   V(s1) = 8.572   V(s2) = 6.323

iter

基于收敛之后的状态值函数，就可以得到最优策略:

$$\pi^*(s) = argmax_a \sum_{s'} P(s' | s,a) \cdot [ r(s,a,s') + \gamma V_{i}(s')] = argmax_a Q_i(s,a)$$

In [12]:
def get_optimal_action(mdp, state_values, state, gamma=0.9):
    """ Finds optimal action using formula above. """
    if mdp.is_terminal(state): return None
     
    actions = mdp.get_possible_actions(state)
    
    optimal_action = None
    optimal_action_value = - float("inf")
    for action in actions:
        action_value = get_action_value(mdp, state_values, state, action, gamma)
        if action_value >= optimal_action_value:
            optimal_action_value = action_value
            optimal_action = action
    
    return optimal_action

In [13]:
assert get_optimal_action(mdp, state_values, 's0', gamma) == 'a1'
assert get_optimal_action(mdp, state_values, 's1', gamma) == 'a0'
assert get_optimal_action(mdp, state_values, 's2', gamma) == 'a0'

In [14]:
# Measure agent's average reward

s = mdp.reset()
rewards = []
for _ in range(10000):
    s, r, done, _ = mdp.step(get_optimal_action(mdp, state_values, s, gamma))
    rewards.append(r)
    
print("average reward: ", np.mean(rewards))

assert(0.85 < np.mean(rewards) < 1.0)

average reward:  0.9235
